In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [14]:
from wordle import Wordle

In [2]:
import torch
import os, json, re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from huggingface_hub import login
login(token="INSERT YOUR HF TOKEN HERE")

/home/lucasbandarkar/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/lucasbandarkar/.cache/huggingface/token
Login successful


In [3]:
llama3_8b = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(llama3_8b, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(llama3_8b, 
                                             torch_dtype="auto",
                                             device_map="auto", ).to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

In [5]:
prompt1 = """
Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position (G) an invalid letter (X) or a correct letter present in the word but in a wrong position (Y). After each guess, we get a result

For example:
BAGEL
XXXGX

Here E is a valid letter in the final word and in the correct position.
You will get a list of invalid letters and valid letters.
Do not use invalid letters in the guess word.
The guess word should be a valid english word.
Guess the correct 5 letter word.

Invalid letters: {{$invalid_letters}}
Valid letters: {{$valid_letters}}
Guess word:  """


prompt2 = """
Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position (G) an invalid letter (X) or a correct letter present in the word but in a wrong position (Y). After each guess, we get a result

For example:
BAGEL
XXXGX

Here E is a valid letter in the final word and in the correct position.
You will get a list of previous guesses and its result. You will also get a word that has letters in correct position with unknown letters as '-'
The guess word should be a valid english word.
Guess the correct 5 letter word.

{{$history}}

Valid letters with positions: {{$valid_letter_word}}
Guess word:  """

prompt3 = """
Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position (G) an invalid letter (X) or a correct letter present in the word but in a wrong position (Y). After each guess, we get a result

For example:
BAGEL
XXXGX

Here E is a valid letter in the final word and in the correct position.
Do not use invalid letters in the guess word.
The guess word should be a valid english word.
You will get a list of previous guesses and its result, invalid letters and valid letters.
Write your reasoning for what should be the guess word, then write the guess word in this format: GUESS->
Guess the correct 5 letter word.

{{$history}}

Invalid letters: {{$invalid_letters}}
Valid letters: {{$valid_letters}}
Reasoning:  """

In [17]:
w = Wordle()

In [19]:

messages = [
    {"role": "system", "content": "Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position (G) an invalid letter (X) or a correct letter present in the word but in a wrong position (Y). After each guess, we get a result\nExample #1:"},
    {"role": "user", "content": "Enter your first guess"},
    {"role": "assistant", "content": "BAGEL"},
    {"role": "user", "content": w.convert_to_readable("XXXXX")},
    {"role": "user", "content": "Enter your second guess"},
    {"role": "assistant", "content": "COVER"},
    {"role": "user", "content": w.convert_to_readable("XXXGY")},
    {"role": "user", "content": "Enter your third guess"},
    {"role": "assistant", "content": "THREE"},
    {"role": "user", "content": w.convert_to_readable("GGGGG") + ", Congrats you found the correct word THREE in 3 guesses"},
    {"role": "system", "content": "Here E is a valid letter in the final word and in the correct position.\nDo not use invalid letters in the guess word.\nThe guess word should be a valid english word.\nYou will get a list of previous guesses and its result, invalid letters and valid letters.\nWrite your reasoning for what should be the guess word, then write the guess word in this format: GUESS->\nGuess the correct 5 letter word."},
    {"role": "system", "content": "Example #2:"},
    {"role": "user", "content": "Enter your first guess"},
    {"role": "assistant", "content": "WHILE"},
    {"role": "user", "content": w.convert_to_readable("XXXXX")},
    {"role": "user", "content": "Enter your second guess"},
    {"role": "assistant", "content": "RADIO"},
    {"role": "user", "content": w.convert_to_readable("XYXXX")},
    {"role": "user", "content": "Enter your third guess"},
    {"role": "assistant", "content": "START"},
    {"role": "user", "content": w.convert_to_readable("GGGXX")},
    {"role": "assistant", "content": "STAGE"},
    {"role": "user", "content": w.convert_to_readable("GGGXX")},
    {"role": "assistant", "content": "STAFF"},
    {"role": "user", "content": w.convert_to_readable("GGGGG") + ", Congrats you found the correct word STAFF in 5 guesses"},
    {"role": "user", "content": "Enter your first guess"},
    {"role": "assistant", "content": "Thank"},
    {"role": "user", "content": w.convert_to_readable("GXYYY")},
    {"role": "user", "content": "Enter your second guess"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=5,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


COVER
